# 📚 Mood-Based Book Recommender

## 🎯 Project Overview

An interactive chatbot that recommends books based on your mood or query. It uses the Book-Crossing dataset from Kaggle and performs semantic similarity search using sentence-transformers and FAISS.

## 🚀 Tech Stack

- **Python**
- **Google Colab**
- **KaggleHub** (for direct dataset fetching)
- **Sentence-Transformers** (embeddings)
- **FAISS** (vector similarity search)
- **Gradio** (chatbot interface)

## 📦 Features

- Takes user input describing mood/preferences
- Embeds book metadata (title, author, publisher) into semantic space
- Finds and recommends the most relevant books using vector search
- Friendly conversational chatbot UI using Gradio



In [1]:
!pip install -q kagglehub sentence-transformers faiss-cpu gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.4 MB/s eta 0:00:00


In [2]:
import kagglehub
dataset_path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

In [3]:
import pandas as pd
import os

books_df = pd.read_csv(os.path.join(dataset_path, "Books.csv"), encoding="latin-1")
books_df = books_df[["Book-Title", "Book-Author"]].dropna().drop_duplicates()

# Clean up
books_df["text"] = books_df["Book-Title"] + " by " + books_df["Book-Author"]
books_df = books_df.reset_index(drop=True)

books = books_df["text"].tolist()


/tmp/ipython-input-3-89122454.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv(os.path.join(dataset_path, "Books.csv"), encoding="latin-1")


In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
book_embeddings = model.encode(books, show_progress_bar=True)

# FAISS index for similarity search
index = faiss.IndexFlatL2(book_embeddings.shape[1])
index.add(book_embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7850 [00:00<?, ?it/s]

In [17]:
def recommend_books_from_mood_gradio(message, history):
    try:
        if not message or not isinstance(message, str):
            return "⚠️ Please enter a valid message."

        input_vector = model.encode([message])
        distances, indices = index.search(np.array(input_vector), k=10)
        results = [books[i] for i in indices[0]]

        response = "📚 Here are some book recommendations based on your mood:\n\n"
        for i, book in enumerate(results, 1):
            response += f"{i}. {book}\n"
        return response

    except Exception as e:
        return f"⚠️ Error occurred in Gradio: {str(e)}"


In [20]:
print(recommend_books_from_mood("I'm feeling angry, can you recommend which will make me calm"))


📚 Here are some book recommendations based on your mood:

1. How to Control Your Anger Before It Controls You by Albert Ellis
2. Angry All the Time: An Emergency Guide to Anger Control by Ron Potter-Efron
3. Working With Anger by Thubten Chodron
4. Managing Anger: A Handbook of Proven Techniques (The Emotional First Aid Series for You and Your Loved Ones) by Mitchell Messer
5. Anger Therapy (Elf Self Help) by Lisa Englehardt



In [21]:
import gradio as gr

gr.ChatInterface(
    fn=recommend_books_from_mood_gradio,
    title="📚 Mood-Based Book Recommender",
    description="Tell me how you're feeling or what you’d love to read — I’ll suggest great books!",
).launch()


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2835aacf450ac42f72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
